## Export to ONNX model

In [1]:
import torch
import timm

resnet18 = timm.create_model('resnet18', pretrained=False)
resnet18.load_state_dict(torch.load('models/resnet18_out1000.pt'))
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [2]:
onnx_file_name = 'resnet18_out1000.onnx'
resnet18.eval()
dummy_input = torch.randn(1, 3, 224, 224, requires_grad=True)
torch.onnx.export(resnet18,  # 模型的名称
            dummy_input,  # 一组实例化输入
            'models/' + onnx_file_name,  # 文件保存路径/名称
            export_params=True,  #  如果指定为True或默认, 参数也会被导出. 如果你要导出一个没训练过的就设为 False.
            opset_version=10,          # ONNX 算子集的版本，当前已更新到15
            do_constant_folding=True,  # 是否执行常量折叠优化
            input_names = ['input'],   # 输入模型的张量的名称
            output_names = ['output'], # 输出模型的张量的名称
            # dynamic_axes将batch_size的维度指定为动态，
            # 后续进行推理的数据可以与导出的dummy_input的batch_size不同
            dynamic_axes={'input' : {0 : 'batch_size'},    
                        'output' : {0 : 'batch_size'}}
            )

## Load ONNX model

In [3]:
import onnx
# 我们可以使用异常处理的方法进行检验
onnx_model_path = 'models/resnet18_out1000.onnx'
onnx_model = onnx.load(onnx_model_path)
try:
    # 当我们的模型不可用时，将会报出异常
    onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s"%e)
else:
    # 模型可用时，将不会报出异常，并会输出“The model is valid!”
    print("The model is valid!")

The model is valid!


## ONNX model Inference

In [4]:
import onnxruntime
import torch

onnx_model_path = 'models/resnet18_out1000.onnx'
ort_session = onnxruntime.InferenceSession(onnx_model_path)
sample = torch.randn(1, 3, 224, 224)
ort_inputs = {'input': sample.numpy()}
ort_output = ort_session.run(None, ort_inputs)[0]
ort_output

array([[ 6.52015209e-02,  2.69498777e+00,  2.71334124e+00,
         2.76640153e+00,  4.62710714e+00,  3.53197861e+00,
         3.78600502e+00,  5.71597755e-01, -7.36629725e-01,
        -4.31888461e-01, -2.09437013e-01,  1.31531882e+00,
         6.87272847e-01,  1.75745130e+00,  1.89299452e+00,
         1.23013341e+00, -4.56359237e-01,  9.26326662e-02,
         1.64535213e+00,  6.73142076e-01, -1.47941709e-03,
         4.74811792e-02,  1.78840828e+00,  1.13033271e+00,
        -6.20750338e-02,  2.78140962e-01,  7.04748690e-01,
         6.14604175e-01,  3.00333023e-01,  6.24658227e-01,
         1.42451942e+00,  2.66387284e-01, -6.45652413e-01,
         2.82643223e+00,  3.56874943e+00,  9.63202477e-01,
         5.44722259e-01,  5.86385369e-01,  4.60738182e-01,
         1.82444119e+00,  1.50553620e+00,  9.82414722e-01,
         1.12724483e+00,  4.93858755e-01,  1.94858670e+00,
         3.66044849e-01,  1.75782251e+00, -1.45140839e+00,
         1.55587792e+00,  7.14112163e-01,  2.20425916e+0

In [5]:
ort_session.get_outputs()[0].name

'output'